In [16]:
import time
import datasets
import pandas as pd
import transformers
import tensorflow as tf
from keras import layers
from tensorflow.keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping
import numpy

train_path="data/train.csv"
dev_path="data/dev.csv"

### Pre-processing

In [17]:
# use the tokenizer from DistilRoBERTa
tokenizer = transformers.AutoTokenizer.from_pretrained("distilroberta-base", pad_token_id=0)

def tokenize(examples):
    """Converts the text of each example to "input_ids", a sequence of integers
    representing 1-hot vectors for each token in the text"""
    return tokenizer(examples["text"], truncation=True, max_length=64,
                     padding="max_length")

# load the CSVs into Huggingface datasets to allow use of the tokenizer
hf_dataset = datasets.load_dataset("csv", data_files={
    "train": train_path, "validation": dev_path})

# the labels are the names of all columns except the first
labels = hf_dataset["train"].column_names[1:]

def gather_labels(example):
    """Converts the label columns into a list of 0s and 1s"""
    # the float here is because converting hf to tf data requires a list or array of labels
    return {"labels": [float(example[l]) for l in labels]}

# convert text and labels to format expected by model
hf_dataset = hf_dataset.map(gather_labels)
hf_dataset = hf_dataset.map(tokenize, batched=True)

Found cached dataset csv (/home/reed/.cache/huggingface/datasets/csv/default-0989c6c4d599a70a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /home/reed/.cache/huggingface/datasets/csv/default-0989c6c4d599a70a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-4172f109611e29af.arrow
Loading cached processed dataset at /home/reed/.cache/huggingface/datasets/csv/default-0989c6c4d599a70a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-4f568e17022e9997.arrow
Loading cached processed dataset at /home/reed/.cache/huggingface/datasets/csv/default-0989c6c4d599a70a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-cdee06b4baee8bb9.arrow
Loading cached processed dataset at /home/reed/.cache/huggingface/datasets/csv/default-0989c6c4d599a70a/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-fe796d7e8ab97072.arrow


### Convert to TF format

In [18]:
# split train and val into their own objects
hf_train = hf_dataset['train']
hf_dev = hf_dataset['validation']

# rename column for embeddings layer
hf_train = hf_train.rename_column("input_ids", "embedding_inputs")
hf_dev = hf_dev.rename_column("input_ids", "embedding_inputs")

# convert Huggingface datasets to Tensorflow datasets
train_dataset = hf_train.to_tf_dataset(
    columns="embedding_inputs",
    label_cols="labels",
    batch_size=32,
    shuffle=True)
dev_dataset = hf_dev.to_tf_dataset(
    columns="embedding_inputs",
    label_cols="labels",
    batch_size=32)

### Model

In [20]:
# define grid search parameters and loop

# define a model with a single fully connected layer
model = tf.keras.Sequential()
model.add(layers.Embedding(
    input_dim=tokenizer.vocab_size,
    output_dim=16,
    mask_zero=True))
model.add(layers.Bidirectional(layers.GRU(64)))
# final processing with a dense RELU layer
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(
    units=len(labels),
    activation='sigmoid'))

# specify compilation hyperparameters
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.001),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=[tf.keras.metrics.F1Score(average="micro", threshold=0.5)])

# set time for run time and model naming
start_time = time.time()
desc = "weights-and-masks"
model_name = f"{desc}_time_{start_time}"

# fit the model to the training data, monitoring F1 on the dev data
print(model_name)
model.fit(
    train_dataset,
    epochs=50,
    validation_data=dev_dataset,
    callbacks=[
        ModelCheckpoint(
            filepath=f"checkpoints/{model_name}",
            monitor="val_f1_score",
            mode="max",
            save_best_only=True),
        TensorBoard(
            log_dir=f"logs/{model_name}"),
        EarlyStopping(
            monitor="val_f1_score",
            min_delta=0.25,
            patience=5,
            start_from_epoch=2)
        ])

time_elapsed = (time.time() - start_time) / 60
print(f"Time Elapsed: {time_elapsed} min.")

weights-and-masks_time_1701720492.9127078
Epoch 1/50
788/788 [==============================] - ETA: 0s - loss: 0.1793 - f1_score: 0.3755

### Predict

In [ ]:
# load the saved model
model_path="checkpoints/batch-size_32_time_1701555125.2780304"
model = tf.keras.models.load_model(model_path)
df = pd.read_csv(dev_path)

# generate predictions from model (on the tf version of validation data)
predictions = numpy.where(model.predict(dev_dataset) > 0.5, 1, 0)

# assign predictions to label columns in Pandas data frame
df.iloc[:, 1:] = predictions
print(df.head())

# write the Pandas dataframe to a zipped CSV file
df.to_csv("submission.zip", index=False, compression=dict(
    method='zip', archive_name=f'submission.csv'))

